In [ ]:
# cargo_schedule_optimization.ipynb

# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import linprog

# Step 2: Load data
cargo_data = pd.read_csv('data/Cargo_Schedule.csv')
weather_data = pd.read_csv('data/Weather_Data.csv')
road_network_data = pd.read_csv('data/Road_Network.csv')

# Step 3: Data Preprocessing (Handling missing values, converting columns, etc.)
cargo_data['schedule_time'] = pd.to_datetime(cargo_data['schedule_time'])
weather_data['timestamp'] = pd.to_datetime(weather_data['timestamp'])

# Step 4: Analyze cargo scheduling and create optimization model
# Simple optimization example: Minimize delays for cargo deliveries considering weather and road networks.

# Create a graph of the road network
G = nx.DiGraph()
for index, row in road_network_data.iterrows():
    G.add_edge(row['start_location'], row['end_location'], weight=row['estimated_travel_time_mins'])

# Calculate travel time from origin to destination
def calculate_travel_time(origin, destination):
    try:
        shortest_path = nx.shortest_path(G, source=origin, target=destination, weight='weight')
        total_time = sum(G[u][v]['weight'] for u, v in zip(shortest_path[:-1], shortest_path[1:]))
        return total_time
    except nx.NetworkXNoPath:
        return np.inf  # No path found

# Step 5: Optimize cargo schedules
# Objective: Minimize delays for each cargo (taking into account estimated travel time and weather conditions)

def optimize_schedule(cargo_data, weather_data):
    delays = []
    for index, row in cargo_data.iterrows():
        origin = row['origin']
        destination = row['destination']
        scheduled_time = row['schedule_time']
        
        # Simulate weather impact on travel time (e.g., rain can increase travel time)
        weather_conditions = weather_data[weather_data['location'] == origin]
        if not weather_conditions.empty:
            weather_factor = 1 + weather_conditions['precipitation'].iloc[0] * 0.5
        else:
            weather_factor = 1

        # Calculate travel time
        travel_time = calculate_travel_time(origin, destination)
        adjusted_travel_time = travel_time * weather_factor
        
        # Calculate the delay by comparing actual travel time to the scheduled time
        actual_arrival_time = scheduled_time + pd.Timedelta(minutes=adjusted_travel_time)
        delay = max(0, (actual_arrival_time - scheduled_time).total_seconds() / 60)  # in minutes
        delays.append(delay)
    
    cargo_data['delay'] = delays
    return cargo_data

optimized_cargo_data = optimize_schedule(cargo_data, weather_data)

# Step 6: Visualize optimization results
plt.figure(figsize=(10,6))
plt.bar(optimized_cargo_data['cargo_id'], optimized_cargo_data['delay'])
plt.xlabel('Cargo ID')
plt.ylabel('Delay (minutes)')
plt.title('Cargo Schedule Delays After Optimization')
plt.show()

# Step 7: Optimization using linear programming (Example for minimizing total delays)
# Define the linear programming problem
c = optimized_cargo_data['delay'].values  # coefficients (delays)
A = np.ones((1, len(c)))  # constraint matrix (sum of delays)
b = [1000]  # total delay should not exceed 1000 minutes

# Solve the optimization problem
result = linprog(c, A_ub=A, b_ub=b, method='simplex')
optimized_delays = result.fun
print(f"Optimized total delay: {optimized_delays:.2f} minutes")
